# Trabalho Prático 2
## Introdução à Computação Visual
#### João Antonio Oliveira Pedrosa - 2019006752

### Calibração da câmera

A calibração da câmera foi feita por meio de imagens obtidas a partir do vídeo de entrada original utilizando o software MatLab.

|                    | Valores Intrínsecos |                 |
|:------------------:|:-------------------:|-----------------|
| 412.5295472  |          0         |  315.1829856  |
| 0                | 409.1999765      | 225.0685988 |
| 0                | 0                  | 1              |

In [1]:
import cv2
import numpy as np
import time
import math

from OpenGL.GL import *
from OpenGL.GLU import *
from OpenGL.GLUT import *

from objloader import *
from PIL import Image

import pygame

pygame 2.1.2 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### Constantes

IMAGE define o alvo a ser localizado.

TEXTURE define a textura à ser aplicada.

VIDEO define o vídeo de entrada.

PERSISTENCE define por quanto tempo um alvo será mantido. Diminuir esse valor pode fazer o Pikachu flickar e aumentar pode fazer ele não ficar tão consistentemente em cima dos alvos.

Valores possíveis para SPIN:

* ccw  - O pikachu irá rodar em sentido anti-horário
* cw   - O pikachu irá rodar em sentido horário
* none - O pikachu não irá rodar

In [2]:
IMAGE       = "./alvo.jpg"
TEXTURE     = "Pikachu.obj"
VIDEO       = "./entrada.mp4"
SPIN        = "cw"
PERSISTENCE = 0.1

#### Função para detecção dos alvos no Vídeo

A função recebe um frame do vídeo e retorna as coordenadas dos alvos localizados no frame.

Utilizamos as funções do Cv2 para Binarizar a imagem e encontrar as bordas no formato que desejamos.

Feito isso, comparamos todas as possíveis rotações do alvo e escolhemos a que gera o menor erro quadrado. Apesar do Pikachu estar rodando, é bom saber a orientação do alvo para manter a rotação consistente.

Além disso, o array de coordenadas também mantém o tempo em que a detecção foi feita, assim, é possível persistir cada alvo por PERSISTENCE segundos.

Tal modificação é feita para reduzir o efeito de "flickering" causado nos objetos por pequenas falhas na detecção do alvo entre frames.

In [3]:
# Pearson Coefficient
def pearson(X, Y):
    std = X.std() * Y.std()
    if std == 0:
        return 0
    
    return np.mean((X - X.mean()) * (Y - Y.mean()))/std

last_targets = []

def getCoords(img):
    size = 100
    global target
    global last_targets
    global PERSISTENCE
    max_dist = float('-inf')

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, gray = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    targets_values = []
    edges = cv2.Canny(gray, 100, 200)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    
    for contour in contours:
        peri = cv2.arcLength(contour, True)
        guess = cv2.approxPolyDP(contour, 12, True)

        if len(guess) == 4:
            indices = None
            coords  = None
            
            dst = np.array([
                [0, 0], 
                [0, size], 
                [size, size], 
                [size, 0]], 
                dtype="float32")
            
            # Determining minimum error among all 4 rotations
            for i in range(4):
                X           = np.array(guess.reshape(4,2), dtype="float32")
                tf_matrix   = cv2.getPerspectiveTransform(X, dst[np.arange(4)])
                warp        = cv2.warpPerspective(gray, tf_matrix, (size, size))
                ret, binary = cv2.threshold(warp, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                
                # Calculating Pearson
                dist = pearson(binary, target)
                if dist > 0.65 and dist > max_dist:
                    max_dist = dist
                    indices = np.arange(4)
                    coords = X
                    break
                    
                target = cv2.rotate(target, cv2.ROTATE_90_CLOCKWISE)
            if coords is not None:
                targets_values.append((coords, indices, time.time()))
    
    # Persisting coordinates so Pikachu doesn't flick
    for cords, indices, t in last_targets:
        found = False
        if time.time() - t < PERSISTENCE:
            for cords2, indices2, _ in targets_values:
                sDist = 0
                for i in range(len(cords)):
                    sDist += (abs(cords[i][0] - cords[i][1]) ** 2) + (abs(cords[i][1] - cords2[i][1]) ** 2)
                if(sDist < 100000) or len(targets_values) == len(last_targets):
                    found = True
            if not found:
                targets_values.append((cords, indices, t))
    
    last_targets = targets_values
                
    return targets_values

A partir daqui, alguns trechos são muito relacionados ao funcionamento do OpenGL então irei apenas explicar o propósito da função, sem entrar em muitos detalhes.

#### Início do Framework

Baseado no código esqueleto, foi adicionada a funcionalidade de utilizar um frame como o background.

In [4]:
def initOpenGL(img):
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    
    background = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    background = cv2.flip(background, 0)
    height, width, channels = background.shape
    background = np.frombuffer(background.tostring(), dtype=background.dtype, count=height * width * channels)
    background.shape = (height, width, channels)
    
    glEnable(GL_TEXTURE_2D)
    glBindTexture(GL_TEXTURE_2D, p_texture)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glDepthMask(GL_FALSE)
    glMatrixMode(GL_PROJECTION)
    
    glPushMatrix()
    glLoadIdentity()
    gluOrtho2D(0, width, 0, height)
    glMatrixMode(GL_MODELVIEW)
    glBindTexture(GL_TEXTURE_2D, p_texture)
    glTexImage2D(GL_TEXTURE_2D, 0, 3, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, background)
    glPushMatrix()
    
    glBegin(GL_QUADS)
    glTexCoord2i(0, 0); glVertex2i(0, 0)
    glTexCoord2i(1, 0); glVertex2i(width, 0)
    glTexCoord2i(1, 1); glVertex2i(width, height)
    glTexCoord2i(0, 1); glVertex2i(0, height)
    glEnd()
    
    glPopMatrix()
    glMatrixMode(GL_PROJECTION)
    glPopMatrix()
    glMatrixMode(GL_MODELVIEW)
    glDepthMask(GL_TRUE)
    glDisable(GL_TEXTURE_2D)

### Função de Renderização

Analisa cada frame do código, faz a chamada para detectar os alvos, renderiza os objetos no local correto e implementa a rotação.

In [5]:
def glutCallback():
    glEnable(GL_TEXTURE_2D)
    glMatrixMode(GL_MODELVIEW)
    
    global SPIN
    _, img = video.read()
    initOpenGL(img)
    targets = getCoords(img)
    
    if   SPIN == "ccw":
        degree = time.time() * 100 % 360
    elif SPIN == "cw":
        degree = -time.time() * 100 % 360
    else:
        degree = 0
    
    for coords, i, _ in targets:
        img_coords = np.array(coords, dtype="float32")
        obj_coords = np.array([[-1, -1, 1], [ 1, -1, 1], [ 1,  1, 1], [-1,  1, 1]], dtype="float32")
        _, rvecs, tvecs = cv2.solvePnP(obj_coords[i], img_coords, intrinsic, distortion_coefficients)
        rotation_matrix, _ = cv2.Rodrigues(rvecs)
        mat = np.transpose(np.array([
            [ rotation_matrix[0][0],  rotation_matrix[0][1],  rotation_matrix[0][2],  tvecs[0]], 
            [-rotation_matrix[1][0], -rotation_matrix[1][1], -rotation_matrix[1][2], -tvecs[1]], 
            [-rotation_matrix[2][0], -rotation_matrix[2][1], -rotation_matrix[2][2], -tvecs[2]], 
            [           0.0,            0.0,            0.0,       1.0]]))
        
        glBindTexture(GL_TEXTURE_2D, p_texture)
        glEnable(GL_TEXTURE_2D)
        glLoadMatrixd(mat)
        glRotatef(degree, 0, 0, 1)
        glCallList(obj.gl_list)
        
    glutSwapBuffers()

#### Função Principal

##### Configuração

* Parâmetros Intrínsecos
* Tamanho das Imagens
* Configuração da Textura
* Carregamento dos Arquivos

##### Loop Principal
Realiza as chamadas para iniciar a aplicação

In [ ]:
if __name__ == '__main__':
    
    # Configurações Iniciais
    width, height = 640, 480
    glutInit()
    glutInitDisplayMode(GLUT_RGBA | GLUT_DOUBLE)
    glutInitWindowSize(width, height)
    glutSetOption(GLUT_ACTION_ON_WINDOW_CLOSE, GLUT_ACTION_CONTINUE_EXECUTION)
    window = glutCreateWindow(b'ICV - TP02')
    glMatrixMode(GL_PROJECTION)  
    fovy = 60
    gluPerspective(fovy, width/height, 0.01, 100.0)
    
    # Definição de Valores Globais
    global intrinsic, distortion_coefficients
    intrinsic = np.array([[412.529,    0,    315.1829], 
                          [0,      409.1999, 225.0685988], 
                          [0,          0,       1]])
    distortion_coefficients = np.array([0.07271, -0.03414, -0.00112, -0.00513, 0.00000]) 
    
    global target
    target = cv2.imread(IMAGE)
    target = cv2.resize(target, (100, 100), interpolation = cv2.INTER_AREA)
    target = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)
    _, target = cv2.threshold(target, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    
    global obj, p_texture
    obj = OBJ(TEXTURE, swapyz=True)
    glEnable(GL_TEXTURE_2D)
    p_texture = glGenTextures(1)
    
    global video
    video = cv2.VideoCapture(VIDEO)
    
    # Loop Principal
    glutDisplayFunc(glutCallback)
    glutIdleFunc(glutPostRedisplay)
    glutMainLoop()
    
    video.release()

Unable to load numpy_formathandler accelerator from OpenGL_accelerate
/tmp/ipykernel_137889/2937966573.py:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  background = np.frombuffer(background.tostring(), dtype=background.dtype, count=height * width * channels)
/tmp/ipykernel_137889/3463177296.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mat = np.transpose(np.array([
Exception ignored on calling ctypes callback function: <function glutCallback at 0x7fe915592b80>
Traceback (most recent call last):
  File "/tmp/ipykernel_137889/3463177296.py", line 7, in glutCallback
  File "/tmp/ipykernel_137889/2937966573.py", line 5, in initOpenGL
cv2.error: OpenCV(4.6.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_s